In [ ]:
import os
import tensorflow as tf
import numpy as np
from numpy import load

from matplotlib import pyplot
from matplotlib.image import imread

from keras.models import Sequential
from keras.layers import Flatten, Dense, BatchNormalization, Dropout
from keras.optimizers import SGD, Adadelta, Adam, Adagrad
from keras.initializers import RandomNormal
from keras.initializers import HeNormal, GlorotUniform
import tensorflow as tf
import datetime
from tensorboard import notebook
import tensorflow_addons as tfa
from keras import backend as K
from sklearn.model_selection import train_test_split

In [ ]:
#pip install keras
#!pip install tensorflow-addons
#!pip install -q pyyaml h5py 
%load_ext tensorboard

notebook.list()

### Loading dataset, reshaping and saving to .npy
#### Using kaggle.com cats_vs_dogs dataset, only "train" folder is used

In [ ]:
# loading dataset, reshaping and saving to 
from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
# define location of dataset
folder = 'train/'
photos, labels = list(), list()
# enumerate files in the directory
for file in listdir(folder):
  # determine class
    output = 0.0
    if file.startswith('cat'):
        output = 1.0
  # load image
    photo = load_img(folder + file, target_size=(200, 200))
  # convert to numpy array
    photo = img_to_array(photo)
  # store
    photos.append(photo)
    labels.append(output)
# convert to a numpy arrays
photos = asarray(photos)
labels = asarray(labels)
print(photos.shape, labels.shape)
# save the reshaped photos
save('dogs_vs_cats_photos.npy', photos)
save('dogs_vs_cats_labels.npy', labels)

### Loading .npy in variables (12 GB RAM required)

In [ ]:
### loading dataset into variables in one run + normalization
train_set, valid_set, train_labels, valid_labels = train_test_split(load('dogs_vs_cats_photos.npy').reshape(25000, 120000) / 255.0, load('dogs_vs_cats_labels.npy'), train_size=0.8, test_size=0.2)

### Tensorboard init + short info about the structure of all available models

In [ ]:
### MODEL INFO
# model1 - 1 sigmoid norm init + SGD
# model2 - 1 relu He init + 1 sigmoid norm init + SGD
# model3 - 3 relu He init + sigmoid He + SGD
# model4 - 4 tanh GlorotUniform init + sigmoid GlorotUniform init + SGD
# model5 - 4 elu He init + sigmoid He init + SGD
# model6 - 4 elu He init + sigmoid He init + Adadelta
# model7 - 4 elu He init + sigmoid He init + Adam
# model8 - 4 elu He init + sigmoid He init + Adagrad
# model11 - 5 elu He init + sigmoid He init + Adadelta
# model12 - 4 elu He init + sigmoid He init + Adadelta + Batch norm
# model13 - 4 elu He init + sigmoid He init + Adadelta + Dropout + Batch norm
# model14 - 4 leaky relu He init + sigmoid He init + Adadelta
# model15 - 4 param relu He init + sigmoid He init + Adadelta
# model16 - 4 elu He init + sigmoid He init + Adadelta + Dropout + Batch norm + l1 l2 + max norm + Early Stop
# model17 - 4 leaky relu He init + sigmoid He init + Adadelta + Batch norm + Dropout + Early Stop
# model18 - 4 leaky relu He init + sigmoid He init + Adadelta + Batch norm + Dropout + l1 l2 + Early Stop

%tensorboard --logdir logs --host localhost

### Custom metric F1

In [ ]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

### Model 1 - sigmoid norm init + SGD

In [ ]:
def define_model():     ### sigmoid only
    model = Sequential()
    model.add(Dense(1, activation='sigmoid', kernel_initializer=RandomNormal(mean=0., stddev=1.)))
    # compile model
    opt = SGD()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', 'AUC', f1])
    return model

In [ ]:
### model 1, simplest 1 sigmoid, normal init
model = define_model()
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

log_dir = "logs/model1/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
model.fit(train_set, train_labels, validation_data = (valid_set, valid_labels), epochs=20, callbacks=[cp_callback, tensorboard_callback])

In [ ]:
### model 1
model.summary()

In [ ]:
### saving final model 1 
#!mkdir -p saved_model
model.save('saved_model/model1')

### Model 2 - 1 relu He init + 1 sigmoid norm init + SGD

In [ ]:
def define_model2():     ### one relu 
    model = Sequential()
    model.add(Dense(100, activation='relu', kernel_initializer=HeNormal()))
    model.add(Dense(1, activation='sigmoid', kernel_initializer=RandomNormal(mean=0., stddev=1.)))
    # compile model
    opt = SGD()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', 'AUC', f1])
    return model

In [ ]:
### model 2, 1 relu + sigmoid, He + normal initializer
model2 = define_model2()
checkpoint_path2 = "training_2/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path2)
cp_callback2 = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path2,
                                                 save_weights_only=True,
                                                 verbose=1)

log_dir2 = "logs/model2/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback2 = tf.keras.callbacks.TensorBoard(log_dir=log_dir2, histogram_freq=1)

In [ ]:
model2.fit(train_set, train_labels, validation_data = (valid_set, valid_labels), epochs=20, callbacks=[cp_callback2, tensorboard_callback2])

In [ ]:
model2.summary()

In [ ]:
model2.save('saved_model/model2')

### Model 3 - 3 relu He init + sigmoid He + SGD

In [ ]:
def define_model3():     ### 3 relu
    model = Sequential()
    model.add(Dense(256, activation='relu', kernel_initializer=HeNormal()))
    model.add(Dense(128, activation='relu', kernel_initializer=HeNormal()))
    model.add(Dense(64, activation='relu', kernel_initializer=HeNormal()))
    model.add(Dense(1, activation='sigmoid', kernel_initializer=HeNormal()))
    # compile model
    opt = SGD()
    model.compile(optimizer=opt, loss='y_p ', metrics=['accuracy', 'AUC', f1])
    return model

In [ ]:
### 3 relu + sigmoid, He init everywhere
model3 = define_model3()
checkpoint_path3 = "training_3/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path3)
cp_callback3 = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path3,
                                                 save_weights_only=True,
                                                 verbose=1)

log_dir3 = "logs/model3/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback3 = tf.keras.callbacks.TensorBoard(log_dir=log_dir3, histogram_freq=1)

In [ ]:
model3.fit(train_set, train_labels, validation_data = (valid_set, valid_labels), epochs=20, callbacks=[cp_callback3, tensorboard_callback3])

In [ ]:
model3.summary()

In [ ]:
model3.save('saved_model/model3')

### Model 4 - 4 tanh GlorotUniform init + sigmoid GlorotUniform init + SGD

In [ ]:
def define_model4():     ### 4 tanh
    model = Sequential()
    #model.add(Dense(1084, kernel_initializer=GlorotUniform(), activation='tanh'))
    #model.add(Dense(408, kernel_initializer=GlorotUniform(), activation='tanh'))
    model.add(Dense(512, kernel_initializer=GlorotUniform(), activation='tanh'))
    model.add(Dense(256, kernel_initializer=GlorotUniform(), activation='tanh'))
    model.add(Dense(128, kernel_initializer=GlorotUniform(), activation='tanh'))
    model.add(Dense(64, kernel_initializer=GlorotUniform(), activation='tanh'))
    model.add(Dense(1, kernel_initializer=GlorotUniform(), activation='sigmoid'))
    opt = SGD()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', 'AUC', f1])
    return model

In [ ]:
### 4 tanh + sigmoid, Glorot init
model4 = define_model4()
checkpoint_path4 = "training_4/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path4)
cp_callback4 = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path4,
                                                 save_weights_only=True,
                                                 verbose=1)

log_dir4 = "logs/model4/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback4 = tf.keras.callbacks.TensorBoard(log_dir=log_dir4, histogram_freq=1)

In [ ]:
model4.fit(train_set, train_labels, validation_data = (valid_set, valid_labels), epochs=20, callbacks=[cp_callback4, tensorboard_callback4])

In [ ]:
model4.summary()

In [ ]:
model4.save('saved_model/model4')

### Model 5 - 4 elu He init + sigmoid He init + SGD

In [ ]:
def define_model5():     ### 4 elu
    model = Sequential()
    model.add(Dense(512, kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dense(256, kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dense(128, kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dense(64, kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dense(1, kernel_initializer=HeNormal(), activation='sigmoid'))
    opt = SGD()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', 'AUC', f1])
    return model

In [ ]:
### 4 elu + sigmoid, He init, SGD opt
model5 = define_model5()
checkpoint_path5 = "training_5/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path5)
cp_callback5 = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path5,
                                                 save_weights_only=True,
                                                 verbose=1)

log_dir5 = "logs/model5/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback5 = tf.keras.callbacks.TensorBoard(log_dir=log_dir5, histogram_freq=1)

In [ ]:
model5.fit(train_set, train_labels, validation_data = (valid_set, valid_labels), epochs=20, callbacks=[cp_callback5, tensorboard_callback5])

In [ ]:
model5.save('saved_model/model5')

### Model 6 - 4 elu He init + sigmoid He init + Adadelta

In [ ]:
def define_model6():     ### 4 elu
    model = Sequential()
    model.add(Dense(512, kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dense(256, kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dense(128, kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dense(64, kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dense(1, kernel_initializer=HeNormal(), activation='sigmoid'))
    opt = Adadelta()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', 'AUC', f1])
    return model

In [ ]:
### 4 elu + adadelta
model6 = define_model6()
checkpoint_path6 = "training_6/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path6)
cp_callback6 = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path6,
                                                 save_weights_only=True,
                                                 verbose=1)

log_dir6 = "logs/model6/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback6 = tf.keras.callbacks.TensorBoard(log_dir=log_dir6, histogram_freq=1)

In [ ]:
model6.fit(train_set, train_labels, validation_data = (valid_set, valid_labels), epochs=20, callbacks=[cp_callback6, tensorboard_callback6])

In [ ]:
model6.save('saved_model/model6')

### Model 7 - 4 elu He init + sigmoid He init + Adam

In [ ]:
def define_model7():     ### 5 elu
    model = Sequential()
    model.add(Dense(512, kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dense(256, kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dense(128, kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dense(64, kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dense(1, kernel_initializer=HeNormal(), activation='sigmoid'))
    opt = Adam()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', 'AUC', f1])
    return model

In [ ]:
### 4 elu + adam
model7 = define_model7()
checkpoint_path7 = "training_7/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path7)
cp_callback7 = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path7,
                                                 save_weights_only=True,
                                                 verbose=1)

log_dir7 = "logs/model7/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback7 = tf.keras.callbacks.TensorBoard(log_dir=log_dir7, histogram_freq=1)

In [ ]:
model7.fit(train_set, train_labels, validation_data = (valid_set, valid_labels), epochs=20, callbacks=[cp_callback7, tensorboard_callback7])

In [ ]:
model7.save('saved_model/model7')

### Model 8 - 4 elu He init + sigmoid He init + Adagrad

In [ ]:
def define_model8():     ### 5 elu
    model = Sequential()
    model.add(Dense(512, kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dense(256, kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dense(128, kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dense(64, kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dense(1, kernel_initializer=HeNormal(), activation='sigmoid'))
    opt = Adagrad()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', 'AUC', f1])
    return model

In [ ]:
### 4 elu + adagrad
model8 = define_model8()
checkpoint_path8 = "training_8/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path8)
cp_callback8 = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path8,
                                                 save_weights_only=True,
                                                 verbose=1)

log_dir8 = "logs/model8/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback8 = tf.keras.callbacks.TensorBoard(log_dir=log_dir8, histogram_freq=1)

In [ ]:
model8.fit(train_set, train_labels, validation_data = (valid_set, valid_labels), epochs=20, callbacks=[cp_callback8, tensorboard_callback8])

In [ ]:
model8.save('saved_model/model8')

### Model 11 - 5 elu He init + sigmoid He init + Adadelta

In [ ]:
def define_model11():     ### 5 elu
    model = Sequential()
    model.add(Dense(1024, kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dense(512, kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dense(256, kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dense(128, kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dense(64, kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dense(1, kernel_initializer=HeNormal(), activation='sigmoid'))
    opt = Adadelta()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', 'AUC', f1])
    return model

In [ ]:
### 4 elu + adadelta
model11 = define_model11()
checkpoint_path11 = "training_11/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path11)
cp_callback11 = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path11,
                                                 save_weights_only=True,
                                                 verbose=1)

log_dir11 = "logs/model11/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback11 = tf.keras.callbacks.TensorBoard(log_dir=log_dir11, histogram_freq=1)

In [ ]:
model11.fit(train_set, train_labels, validation_data = (valid_set, valid_labels), epochs=20, callbacks=[cp_callback11, tensorboard_callback11])

In [ ]:
model11.save('saved_model/model11')

In [ ]:
model11.summary()

### Model 12 - 4 elu He init + sigmoid He init + Adadelta + Batch norm

In [ ]:
def define_model12():     ### 5 elu
    model = Sequential()
    #model.add(Dense(1024, kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dense(512, kernel_initializer=HeNormal(), activation='elu'))
    model.add(BatchNormalization())
    model.add(Dense(256, kernel_initializer=HeNormal(), activation='elu'))
    model.add(BatchNormalization())
    model.add(Dense(128, kernel_initializer=HeNormal(), activation='elu'))
    model.add(BatchNormalization())
    model.add(Dense(64, kernel_initializer=HeNormal(), activation='elu'))
    model.add(BatchNormalization())
    model.add(Dense(1, kernel_initializer=HeNormal(), activation='sigmoid'))
    opt = Adadelta()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', 'AUC', f1])
    return model

In [ ]:
### 4 elu + adadelta * batch norm
model12 = define_model12()
checkpoint_path12 = "training_12/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path12)
cp_callback12 = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path12,
                                                 save_weights_only=True,
                                                 verbose=1)

log_dir12 = "logs/model12/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback12 = tf.keras.callbacks.TensorBoard(log_dir=log_dir12, histogram_freq=1)

In [ ]:
model12.fit(train_set, train_labels, validation_data = (valid_set, valid_labels), epochs=20, callbacks=[cp_callback12, tensorboard_callback12])

In [ ]:
model12.save('saved_model/model12')

### Model 13 - 4 elu He init + sigmoid He init + Adadelta + Dropout + Batch norm

In [ ]:
def define_model13():     ### 5 elu
    model = Sequential()
    #model.add(Dense(1024, kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dense(512, kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dropout(.2))
    model.add(BatchNormalization())
    model.add(Dense(256, kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dropout(.2))
    model.add(BatchNormalization())
    model.add(Dense(128, kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dropout(.2))
    model.add(BatchNormalization())
    model.add(Dense(64, kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dropout(.2))
    model.add(BatchNormalization())
    model.add(Dense(1, kernel_initializer=HeNormal(), activation='sigmoid'))
    opt = Adadelta()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', 'AUC', f1])
    return model

In [ ]:
### 4 elu + adadelta * batch norm
model13 = define_model13()
checkpoint_path13 = "training_13/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path13)
cp_callback13 = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path13,
                                                 save_weights_only=True,
                                                 verbose=1)

log_dir13 = "logs/model13/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback13 = tf.keras.callbacks.TensorBoard(log_dir=log_dir13, histogram_freq=1)

early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)

In [ ]:
model13.fit(train_set, train_labels, validation_data = (valid_set, valid_labels), epochs=30, callbacks=[cp_callback13, tensorboard_callback13, early_stop_callback])

In [ ]:
model13.save('saved_model/model13')

### Model 14 - 4 leaky relu He init + sigmoid He init + Adadelta

In [ ]:
def define_model14():     ### 4 elu
    model = Sequential()
    model.add(Dense(512, kernel_initializer=HeNormal(), activation=tf.keras.layers.LeakyReLU()))
    model.add(Dense(256, kernel_initializer=HeNormal(), activation=tf.keras.layers.LeakyReLU()))
    model.add(Dense(128, kernel_initializer=HeNormal(), activation=tf.keras.layers.LeakyReLU()))
    model.add(Dense(64, kernel_initializer=HeNormal(), activation=tf.keras.layers.LeakyReLU()))
    model.add(Dense(1, kernel_initializer=HeNormal(), activation='sigmoid'))
    opt = Adadelta()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', 'AUC', f1])
    return model

In [ ]:
### 4 elu + adadelta
model14 = define_model14()
checkpoint_path14 = "training_14/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path14)
cp_callback14 = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path14,
                                                 save_weights_only=True,
                                                 verbose=1)

log_dir14 = "logs/model14/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback14 = tf.keras.callbacks.TensorBoard(log_dir=log_dir14, histogram_freq=1)

In [ ]:
model14.fit(train_set, train_labels, validation_data = (valid_set, valid_labels), epochs=20, callbacks=[cp_callback14, tensorboard_callback14])

In [ ]:
model14.save('saved_model/model14')

### Model 15 - 4 param relu He init + sigmoid He init + Adadelta

In [ ]:
def define_model15():     ### 4 elu
    model = Sequential()
    model.add(Dense(512, kernel_initializer=HeNormal(), activation=tf.keras.layers.PReLU()))
    model.add(Dense(256, kernel_initializer=HeNormal(), activation=tf.keras.layers.PReLU()))
    model.add(Dense(128, kernel_initializer=HeNormal(), activation=tf.keras.layers.PReLU()))
    model.add(Dense(64, kernel_initializer=HeNormal(), activation=tf.keras.layers.PReLU()))
    model.add(Dense(1, kernel_initializer=HeNormal(), activation='sigmoid'))
    opt = Adadelta()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', 'AUC', f1])
    return model

In [ ]:
### 4 elu + adadelta
model15 = define_model15()
checkpoint_path15 = "training_15/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path15)
cp_callback15 = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path15,
                                                 save_weights_only=True,
                                                 verbose=1)

log_dir15 = "logs/model15/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback15 = tf.keras.callbacks.TensorBoard(log_dir=log_dir15, histogram_freq=1)

In [ ]:
model15.fit(train_set, train_labels, validation_data = (valid_set, valid_labels), epochs=20, callbacks=[cp_callback15, tensorboard_callback15])

In [ ]:
model15.save('saved_model/model15')

### Model 16 - 4 elu He init + sigmoid He init + Adadelta + Dropout + Batch norm + l1 l2 + max norm + Early Stop

In [ ]:
def define_model16():     ### 5 elu
    model = Sequential()
    #model.add(Dense(1024, kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dense(512, kernel_regularizer='l1_l2', kernel_constraint=tf.keras.constraints.MaxNorm(max_value=2, axis=0), kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dropout(.2))
    model.add(BatchNormalization())
    model.add(Dense(256, kernel_regularizer='l1_l2', kernel_constraint=tf.keras.constraints.MaxNorm(max_value=2, axis=0), kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dropout(.2))
    model.add(BatchNormalization())
    model.add(Dense(128, kernel_regularizer='l1_l2', kernel_constraint=tf.keras.constraints.MaxNorm(max_value=2, axis=0), kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dropout(.2))
    model.add(BatchNormalization())
    model.add(Dense(64, kernel_regularizer='l1_l2', kernel_constraint=tf.keras.constraints.MaxNorm(max_value=2, axis=0), kernel_initializer=HeNormal(), activation='elu'))
    model.add(Dropout(.2))
    model.add(BatchNormalization())
    model.add(Dense(1, kernel_initializer=HeNormal(), activation='sigmoid'))
    opt = Adadelta()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', 'AUC', f1])
    return model

In [ ]:
### 4 elu + adadelta * batch norm
model16 = define_model16()
checkpoint_path16 = "training_16/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path16)
cp_callback16 = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path16,
                                                 save_weights_only=True,
                                                 verbose=1)

log_dir16 = "logs/model16/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback16 = tf.keras.callbacks.TensorBoard(log_dir=log_dir16, histogram_freq=1)

early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)

In [ ]:
model16.fit(train_set, train_labels, validation_data = (valid_set, valid_labels), epochs=5, callbacks=[cp_callback16, tensorboard_callback16, early_stop_callback])

In [ ]:
model16.save('saved_model/model16')

### Model 17 - 4 leaky relu He init + sigmoid He init + Adadelta + Batch norm + Dropout + Early Stop

In [ ]:
def define_model17():     ### 5 elu
    model = Sequential()
    #model.add(Dense(1024, kernel_initializer=HeNormal(), activation='elu'))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Dense(512, kernel_initializer=HeNormal(), activation=tf.keras.layers.LeakyReLU()))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Dense(256, kernel_initializer=HeNormal(), activation=tf.keras.layers.LeakyReLU()))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Dense(128, kernel_initializer=HeNormal(), activation=tf.keras.layers.LeakyReLU()))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Dense(64, kernel_initializer=HeNormal(), activation=tf.keras.layers.LeakyReLU()))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Dense(1, kernel_initializer=HeNormal(), activation='sigmoid'))
    opt = Adadelta()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', 'AUC', f1])
    return model

In [ ]:
### 4 elu + adadelta * batch norm
model17 = define_model17()
checkpoint_path17 = "training_17/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path17)
cp_callback17 = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path17,
                                                 save_weights_only=True,
                                                 verbose=1)

log_dir17 = "logs/model17/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback17 = tf.keras.callbacks.TensorBoard(log_dir=log_dir17, histogram_freq=1)

early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)

In [ ]:
model17.fit(train_set, train_labels, validation_data = (valid_set, valid_labels), epochs=10, callbacks=[cp_callback17, tensorboard_callback17, early_stop_callback])

In [ ]:
model17.fit(train_set, train_labels, validation_data = (valid_set, valid_labels), epochs=10, callbacks=[cp_callback17, tensorboard_callback17, early_stop_callback])

In [ ]:
model17.fit(train_set, train_labels, validation_data = (valid_set, valid_labels), epochs=10, callbacks=[cp_callback17, tensorboard_callback17, early_stop_callback])

In [ ]:
model17.save('saved_model/model17')

### Model 18 - 4 leaky relu He init + sigmoid He init + Adadelta + Batch norm + Dropout + l1 l2 + Early Stop

In [ ]:
def define_model18():     ### 5 elu
    model = Sequential()
    #model.add(Dense(1024, kernel_initializer=HeNormal(), activation='elu'))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Dense(512, kernel_regularizer='l1_l2', kernel_initializer=HeNormal(), activation=tf.keras.layers.LeakyReLU()))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Dense(256, kernel_regularizer='l1_l2', kernel_initializer=HeNormal(), activation=tf.keras.layers.LeakyReLU()))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Dense(128, kernel_regularizer='l1_l2', kernel_initializer=HeNormal(), activation=tf.keras.layers.LeakyReLU()))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Dense(64, kernel_regularizer='l1_l2', kernel_initializer=HeNormal(), activation=tf.keras.layers.LeakyReLU()))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Dense(1, kernel_initializer=HeNormal(), activation='sigmoid'))
    opt = Adadelta()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', 'AUC', f1])
    return model

In [ ]:
### 4 elu + adadelta * batch norm
model18 = define_model18()
checkpoint_path18 = "training_18/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path18)
cp_callback18 = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path18,
                                                 save_weights_only=True,
                                                 verbose=1)

log_dir18 = "logs/model18/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback18 = tf.keras.callbacks.TensorBoard(log_dir=log_dir18, histogram_freq=1)

early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)

In [ ]:
model18.fit(train_set, train_labels, validation_data = (valid_set, valid_labels), epochs=5, callbacks=[cp_callback18, tensorboard_callback18, early_stop_callback])

In [ ]:
model18.fit(train_set, train_labels, validation_data = (valid_set, valid_labels), epochs=5, callbacks=[cp_callback18, tensorboard_callback18, early_stop_callback])

In [ ]:
model18.fit(train_set, train_labels, validation_data = (valid_set, valid_labels), epochs=10, callbacks=[cp_callback18, tensorboard_callback18, early_stop_callback])

### Loading & compiling the model from save

In [ ]:
model_final = tf.keras.models.load_model('saved_model/model17', compile=False)
model_final.compile(optimizer=Adadelta(), loss='binary_crossentropy', metrics=['accuracy', 'AUC', f1])

In [ ]:
model_final.summary()

### Creating small test set 

In [ ]:
# # CREATION OF SMALL TEST SET
# from os import listdir
# from numpy import asarray
# from numpy import save
# from keras.preprocessing.image import load_img
# from keras.preprocessing.image import img_to_array
# # define location of dataset
# folder = 'test/'
# photos, labels = list(), list()
# # enumerate files in the directory
# i=0
# for file in listdir(folder):
#     photo = load_img(folder + file, target_size=(200, 200))
#     # convert to numpy array
#     photo = img_to_array(photo)
#     #store
#     photos.append(photo)
#     i += 1
#     if i > 20:
#         break
# # convert to a numpy arrays
# photos = asarray(photos)
# print(photos.shape)
# # save the reshaped photos
# save('test_photos.npy', photos)

In [ ]:
test_set = load('test_photos.npy').reshape(21, 120000) / 255.0

### Predicting classes for the test images and plotting them

In [ ]:
#predictions for test set
prediction = tf.make_tensor_proto(model_final(test_set[0:20]))
prediction = tf.make_ndarray(prediction)
pyplot.figure(figsize=(15,15))
for i in range(20):
    pyplot.subplot(5,4,i+1)
    pyplot.xticks([])
    pyplot.yticks([])
    pyplot.grid(False)
    pyplot.imshow(test_set[i].reshape(200,200,3))
    if prediction[i][0] >= 0.5:
        pyplot.xlabel("cat")
    else:
        pyplot.xlabel("dog")
pyplot.show()

### Same predictions using predict_classes()

In [ ]:
#predictions for test set
prediction = model_final.predict_classes(test_set)
pyplot.figure(figsize=(15,15))
for i in range(20):
    pyplot.subplot(5,4,i+1)
    pyplot.xticks([])
    pyplot.yticks([])
    pyplot.grid(False)
    pyplot.imshow(test_set[i].reshape(200,200,3))
    if prediction[i][0] >= 0.5:
        pyplot.xlabel("cat")
    else:
        pyplot.xlabel("dog")
pyplot.show()